In [184]:
import pickle
import pandas as pd
import numpy as np
import nltk as nl
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import WordNetLemmatizer
from scipy import spatial
from collections import OrderedDict
nl.download('stopwords')
nl.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [185]:
def read_data():
    dataset = pd.read_csv("Dataset_final_filtered.tsv",sep='\t')
    
    infile = open("vocabulary.pkl",'rb')
    vocab = pickle.load(infile)
    infile.close()
    infile = open("ID_with_tfidf.pkl",'rb')
    ID = pickle.load(infile)
    infile.close()
    return dataset,vocab,ID

dataset,vocabulary,ID = read_data()

dataset.head()

,document_ID,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot_Values,Plot,NumberofPages,Publishing_Date,Characters,Setting,Url
0,0,MARS,NaN,Jasmine Rose,4.38,69,13,"{5: '47', 4: '7', 3: '11', 2: '2', 1: '2'}",❝�� my heart has become a planetand you are th...,NaN,2014,NaN,NaN,https://www.goodreads.com/book/show/23279048-mars
1,1,Black Box,NaN,Cassia Leo,4.02,6244,903,"{5: '2297', 4: '2320', 3: '1181', 2: '345', 1:...",♥️ Three fateful encounters....♥️ Two heart-br...,400.0,February 28th 2014,"['Mikki Gladstone', 'William ""Crush"" Slayer']","['Boston, Massachusetts']",https://www.goodreads.com/book/show/29539518-b...
2,2,Ruin and Rising,The Shadow and Bone Trilogy #3,Leigh Bardugo,4.09,158624,19396,"{5: '62107', 4: '59607', 3: '27962', 2: '6810'...",▶ \nAlternative Cover Edition #1\nThe capital ...,422.0,June 17th 2014,"['Alina Starkov', 'Malyen Oretsev', 'Darkling'...",['Ravka '],https://www.goodreads.com/book/show/14061957-r...
3,3,Shadow and Bone,The Shadow and Bone Trilogy #1,Leigh Bardugo,3.99,313190,32760,"{5: '108466', 4: '121286', 3: '62748', 2: '146...",▶ \nAlternative Cover Edition #1\nSurrounded b...,358.0,June 5th 2012,"['Alina Starkov', 'Malyen Oretsev', 'Darkling'...",['Ravka '],https://www.goodreads.com/book/show/10194157-s...
4,4,An Ember in the Ashes,An Ember in the Ashes #1,Sabaa Tahir,4.27,184959,23147,"{5: '91602', 4: '63735', 3: '21164', 2: '5487'...",▶ \nAlternative Cover Edition #1\nLaia is a sl...,446.0,February 9th 2016,"['Laia', 'Elias Veturius', 'Helene Aquilla', '...",NaN,https://www.goodreads.com/book/show/27774758-a...


In [186]:
lemma = WordNetLemmatizer()
def clean_info (string):
    #first we lower all the words otherwise words such as AND,IS,MY are not consider stopwords 
    tmp = [word.lower() for word in string]
     # filter the stopwords (e.g. 'the', 'my', 'an', ...)
    tmp = [word for word in tmp if not word in stopwords.words("english")]
    #we lemmatize all the words (e.g. 'dirn')
    lemma = WordNetLemmatizer()
    tmp = [lemma.lemmatize(word, pos = "v") for word in tmp]    # v for verbs
    tmp = [lemma.lemmatize(word, pos = "n") for word in tmp]    # n for nouns
    final = ' '.join(tmp)
    return final

def find_set(a,b):
    return set(a) & set(b)

In [201]:
query = input("Please enter the Query: ")
query = clean_info(query.split(' '))
query = query.split(' ')

#Sample Query: * Survival games hunger
print("Query cleaned: ",query)

Please enter the Query: Survival games hunger
Query cleaned:  ['survival', 'game', 'hunger']


In [202]:
lists = {}
docs = {}
for i in query:
    lists[i] = ID.get(vocabulary[i])
    docs[i] = [x[0] for x in lists[i][:]]
    
data = None
flag = True
for i in docs:
    if flag:
        flag = False
        data = docs[i]
        continue
    data = find_set(data,docs[i])
    
data_withVal = {}
for i in lists:
    data_withVal[i] = [x for x in lists[i][:] if x[0] in data]
    
data_flat = []
for x in data_withVal.values():
    data_flat.extend(x)
    
res = OrderedDict()
for v in data:
    val = [cos[1] for cos in [x for x in data_flat if x[0]==v]]
    score = spatial.distance.cosine(len(query),val)
    res[v] = score
    
sorted_books = sorted(res.items(), key=lambda x:x[1],reverse=True)
data_index = []
data_score = []
for i,j in sorted_books:
    data_index.append(i)
    data_score.append(j)

founddata = dataset.iloc[data_index,:][['bookTitle','Plot','Url']][:][0:5]
founddata.insert( 3, "Similarity", data_score[0:5], True) 

founddata

,bookTitle,Plot,Url,Similarity
5513,Inside the Maze Runner: The Guide to the Glade,The first book in James Dashner’s New York Tim...,https://www.goodreads.com/book/show/21965351-i...,0.058119
4979,"The Maze Runner Trilogy (Maze Runner, #1-3)",The perfect gift for fans of The Hunger Games ...,https://www.goodreads.com/book/show/17292676-t...,0.058119
19693,The Hunger Games,"Could you survive on your own in the wild, wit...",https://www.goodreads.com/book/show/2767052-th...,0.015887
